# OpenAI functions

Certain OpenAI models (like gpt-3.5-turbo-0613 and gpt-4-0613) have been fine-tuned to detect when a function should be called and respond with the inputs that should be passed to the function. In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call those functions. The goal of the OpenAI Function APIs is to more reliably return valid and useful function calls than a generic text completion or chat API.

The OpenAI Functions Agent is designed to work with these models.

Install `openai`, `google-search-results` packages which are required as the LangChain packages call them internally.

In [1]:
! pip install openai google-search-results


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


## Initialize tools

We will first create some tools we can use

In [2]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper, SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [3]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
db = SQLDatabase.from_uri("sqlite:///../../../../../notebooks/Chinook.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
    Tool(
        name="FooBar-DB",
        func=db_chain.run,
        description="useful for when you need to answer questions about FooBar. Input should be in the form of a question containing full context",
    ),
]

## Using LCEL

We will first use LangChain Expression Language to create this agent

In [4]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [6]:
from langchain.tools.render import format_tool_to_openai_function

In [7]:
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

In [8]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [9]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [10]:
from langchain.agents import AgentExecutor

In [11]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

['memory', 'callbacks', 'callback_manager', 'verbose', 'tags', 'metadata', 'agent', 'tools', 'return_intermediate_steps', 'max_iterations', 'max_execution_time', 'early_stopping_method', 'handle_parsing_errors', 'trim_intermediate_steps']


In [12]:
agent_executor.invoke(
    {
        "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `Search` with `Leo DiCaprio's girlfriend`


Camila Morrone: 2017 to 2022 These two dated for way longer than usual for Leo, and an anonymous source even told People that Leo was “very in love and serious” and he and Camila had “talked about getting engaged.”
Invoking: `Calculator` with `22^0.43`




> Entering new LLMMathChain chain...
22^0.43```text
22**0.43
```
...numexpr.evaluate("22**0.43")...

Answer: 3.777824273683966
> Finished chain.
Answer: 3.777824273683966Leo DiCaprio's girlfriend is Camila Morrone. Her current age raised to the power of 0.43 is approximately 3.78.

> Finished chain.


{'input': "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
 'output': "Leo DiCaprio's girlfriend is Camila Morrone. Her current age raised to the power of 0.43 is approximately 3.78."}

## Using OpenAIFunctionsAgent

We can now use `OpenAIFunctionsAgent`, which creates this agent under the hood

In [13]:
agent_executor = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

['memory', 'callbacks', 'callback_manager', 'verbose', 'tags', 'metadata', 'agent', 'tools', 'return_intermediate_steps', 'max_iterations', 'max_execution_time', 'early_stopping_method', 'handle_parsing_errors', 'trim_intermediate_steps']


In [14]:
agent_executor.invoke(
    {
        "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `Search` with `{'query': 'Leo DiCaprio girlfriend'}`


In August, Leonardo DiCaprio was seen with Italian model Vittoria Ceretti, despite rumors of a potential relationship with model Gigi Hadid over the past year. New rumors started about Ceretti this summer, when she was seen on a yacht with DiCaprio and other models Meghan Roche and Imaan Hammam.
Invoking: `Calculator` with `{'expression': 'age ^ 0.43', 'variables': {'age': 25}}`




ToolException: Too many arguments to single-input tool Calculator. Args: ['age ^ 0.43', {'age': 25}]